In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

from data_split import data_split_generator
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np
from tqdm import tqdm

In [5]:
def make_x_y(data: pd.DataFrame, month_encoder:OneHotEncoder = None):
    data = data.copy(deep=True)
    data.reset_index(drop=True, inplace=True)
    data.drop(columns=['dates', 'target_dates'], inplace=True)
    if month_encoder is None:
        month_encoder = OneHotEncoder()
        month_encoder.fit(data[['month']])
    month = month_encoder.transform(data[['month']]).toarray()
    month = pd.DataFrame(month, columns=[str(x) for x in month_encoder.categories_[0].tolist()])
    data:pd.DataFrame = pd.concat([data, month], axis = 1)
    Y = data.pop('target_sunspots')
    return data , Y, month_encoder
def get_metric(y_true, y_predict):
    RMSE = np.sqrt(mean_squared_error(y_true, y_predict))
    # MAE = mean_absolute_error(y_true, y_predict)
    # MAPE = mean_absolute_percentage_error(y_true, y_predict)
    return RMSE

# train_x, train_y, month_encoder = make_x_y(train_data)
# train_x

In [8]:
final_metric = []
for j in tqdm(range(120, 133)):
    data = pd.read_csv(f'./data/processed_data_{j}.csv')
    data['target_dates'] = pd.to_datetime(data['target_dates'])
    data_split = data_split_generator(data)
    month_encoder = None
    metrics = []
    for train_data, vali_data in data_split:
        train_x, train_y, month_encoder = make_x_y(train_data, month_encoder)
        rf = RandomForestRegressor(n_jobs=4)
        rf.fit(X=train_x, y=train_y)
        vali_x, vali_y, month_encoder = make_x_y(vali_data, month_encoder)
        predict = rf.predict(vali_x)
        metric = get_metric(vali_y, predict)
        metrics.append(metric)
    metrics = np.mean(metrics)
metrics

100%|██████████| 13/13 [00:05<00:00,  2.22it/s]


47.67446510209752